**[This walkthrough is also available as a Jupyter ipynb Notebook - you can run yourself](notebooks/ex_GettingData.ipynb)**

A frequent question asked is: 'How do I get data?'

Importantly, **we are just working with NodeJS**<br />
albeit within the context within a Jupyter Lab Notebook.

So essentially, any data that you can programmatically access with an [NPM module](https://www.npmjs.com/search?q=keywords:file) is available within a Jupyter Lab notebook.

Example Data Sources:
* Local Files
* APIs
* Cloud Services
* Scraping
* Generating
* etc.

In [1]:
utils = require('jupyter-ijavascript-utils');
cheerio = require('cheerio');
d3 = require('d3');
dotenv = require('dotenv');
['utils', 'cheerio', 'd3', 'dotenv'];

[ 'utils', 'cheerio', 'd3', 'dotenv' ]

# Local Files

While there are many NPM modules available for loading data, we have tried to simplify two main types of files:
* [utils.file.readFile(path)](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.readFile) - to read in Text
* [utils.file.readJSON(path)](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.readJSON) - to read in a file and parse as JSON

Including functions to write files as-well.

* [utils.file.writeFile(path, stringContents)](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.readFile) - to read in Text
* [utils.file.readJSON(path, stringContents)](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.readJSON) - to read in a file and parse as JSON

There are many NPM Modules available for loading different kinds of files.

The jupyter-ijavascript-utils library includes a few simple functions - for requent situations.

* [utils.file.pwd](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.pwd) - prints the current directory Jupyter Lab is looking at

* [utils.file.listFiles](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.listFiles) - to list the files in a directory

* [utils.file.checkFile](https://jupyter-ijavascript-utils.onrender.com/module-file.html#.checkFile) - to return true if a file already exists.

In [3]:
utils.file.writeFile('./credentials.env', `
username="jdoe@example.com"
password="12345"
`)

In [4]:
credentialsPath = `${utils.file.pwd()}/credentials.env`;

if (utils.file.checkFile('./credentials.env')) {
    utils.file.writeFile('./credentials.env', `
username=""
password=""
`);
    
    console.error(`Could not find ${credentialsPath}

We use dotenv to securely store credentials
and require it to access provider XYZ.

an example template was generated for you:
${credentialsPath}

Please update and try running this notebook again.
`);
    throw Error(`credentials file not found:${credentialsPath}
Please complete it and try this command again`);
} else {
    credentials = dotenv.config({ path: credentialsPath }).parsed;
}

if (!credentials.username || !credentials.password) {
    throw Error(`Credentials not provided
${credentialsPath}`);
}

console.log(`credentials loaded for: ${credentials.username}`);

credentials loaded for: jdoe@example.com


In [5]:
simplex = utils.random.simplexGenerator();
['simplex'];

[ 'simplex' ]

In [6]:
2+2

4

# Render out the simplex noise once

In [7]:
utils.ijs.htmlScript({
    debug: true,
    width: 400,
    height: 400,
    utilityFunctions: ({ simplex: utils.random.simplexGenerator }),
    html: '<canvas id="draw-target" />',
    onReady: ({ rootEl, options, utilityFunctions, animate }) => {
        const canvas = rootEl.querySelector('canvas#draw-target');
        
        const noise = utilityFunctions.simplex();
        debugger;
        
        canvas.width = parseInt(options.width);
        canvas.height = parseInt(options.height);

        var ctx = canvas.getContext('2d');

        var image = ctx.createImageData(canvas.width, canvas.height);
        var data = image.data;

        for (var x = 0; x < canvas.width; x++) {
          //if (x % 100 == 0) {
          //  noise.seed(Math.random());
          //}
          for (var y = 0; y < canvas.height; y++) {
            var value = Math.abs(noise.simplex2d(x / 100, y / 100));
            value *= 256;

            var cell = (x + y * canvas.width) * 4;
            data[cell] = data[cell + 1] = data[cell + 2] = value;
            data[cell] += Math.max(0, (25 - value) * 8);
            data[cell + 3] = 255; // alpha.
          }
        }
        
        ctx.fillColor = 'black';
        ctx.fillRect(0, 0, 100, 100);
        ctx.putImageData(image, 0, 0);
    }
})

In [20]:
utils.ijs.htmlScript({
    width: 400,
    height: 400,
    data: { cellSize: 150, textSize: 15 },
    utilityFunctions: ({ simplex: utils.random.simplexGenerator }),
    html: '<canvas id="draw-target" />',
    onReady: ({ rootEl, options, utilityFunctions, data }) => {
        const canvas = rootEl.querySelector('canvas#draw-target');
        
        const noise = utilityFunctions.simplex();
        
        canvas.width = parseInt(options.width);
        canvas.height = parseInt(options.height);

        var ctx = canvas.getContext('2d');

        var image = ctx.createImageData(canvas.width, canvas.height);
        var imageData = image.data;
        
        function setCanvasColor(index, { red, green, blue, alpha }) {
            imageData[index + 0] = red;
            imageData[index + 1] = green;
            imageData[index + 2] = blue;
            imageData[index + 3] = alpha;
        }

        for (var x = 0; x < canvas.width; x++) {
          for (var y = 0; y < canvas.height; y++) {
            var value = noise.simplex2d(x / data.cellSize, y / data.cellSize) * 256;
              
            var cell = (x + y * canvas.width) * 4;

            //-- cells are for each pixel
            //-- and in sets of 4: [red, green, blue, alpha]
            setCanvasColor(
                cell,
                {
                    red: value < 0 ? Math.abs(value) : 0,
                    green: value > 0 ? value : 0,
                    blue: 0,
                    alpha: 255
                }
            );
          }
        }
        
        ctx.fillColor = 'black';
        ctx.fillRect(0, 0, canvas.width, canvas.height);
        ctx.putImageData(image, 0, 0);
        
        ctx.font = '16px sans-serif'
        ctx.fillStyle = 'white';
        ctx.textAlign = 'left';
        ctx.fillText('(0, 0)', 0, data.textSize);
        ctx.fillText('(1, 0)', data.cellSize, data.textSize);
        ctx.fillText('(0, 1)', 0, data.cellSize + data.textSize);
        ctx.fillText('(1, 1)', data.cellSize, data.cellSize + data.textSize);
    }
});

# Do an animation

In [30]:
utils.ijs.htmlScript({
    width: 400,
    height: 400,
    
    data: {
        period: 10000,
        cellSize: 150,
        textSize: 15
    },
    utilityFunctions: ({
        simplex: utils.random.simplexGenerator,
        timePeriod: utils.format.timePeriod
    }),
    html: '<canvas id="draw-target" />',
    onReady: ({ rootEl, options, data, utilityFunctions, animate }) => {
        const canvas = rootEl.querySelector('canvas#draw-target');
        
        const noise = utilityFunctions.simplex();
        
        canvas.width = parseInt(options.width);
        canvas.height = parseInt(options.height);

        var ctx = canvas.getContext('2d');

        var image = ctx.createImageData(canvas.width, canvas.height);
        var imageData = image.data;
        
        function setCanvasColor(index, { red, green, blue, alpha }) {
            imageData[index + 0] = red;
            imageData[index + 1] = green;
            imageData[index + 2] = blue;
            imageData[index + 3] = alpha;
        }

        var height = 0;
                
        function draw() {
          var start = Date.now();
          height = utilityFunctions.timePeriod(data.period);

          for (var x = 0; x < canvas.width; x++) {
            for (var y = 0; y < canvas.height; y++) {
              var value = noise.simplex3d(x / data.cellSize, y / data.cellSize, height) * 256;
                
              var cell = (x + y * canvas.width) * 4;

              //-- cells are for each pixel
              //-- and in sets of 4: [red, green, blue, alpha]
              setCanvasColor(
                cell,
                {
                  red: value < 0 ? Math.abs(value) : 0,
                  green: value > 0 ? value : 0,
                  blue: 0,
                  alpha: 255
                }
              );
            }
          }

          var end = Date.now();

          ctx.fillColor = 'black';
          ctx.fillRect(0, 0, canvas.width, canvas.height);
          ctx.putImageData(image, 0, 0);
        
          ctx.font = '16px sans-serif'
          ctx.fillStyle = 'white';
          ctx.textAlign = 'left';
          ctx.fillText('(0, 0)', 0, data.textSize);
          ctx.fillText('(1, 0)', data.cellSize, data.textSize);
          ctx.fillText('(0, 1)', 0, data.cellSize + data.textSize);
          ctx.fillText('(1, 1)', data.cellSize, data.cellSize + data.textSize);

          animate(draw);
        };
        draw();
    }
})